### 1. 线下
    1.词向量，
        1.1 分词，训练接下来呢？
    2. 句向量
        2.1 分词，训练句子。

### 2.线上
    1.初赛只需要分词就可以了。
    2.复赛    
      2.1 如果线上数据量变大，可能需要先用所有的单词初始化，然后单步训练。

In [1]:
import jieba 
import gensim
import pandas as pd
import numpy as np
import jieba.analyse

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#read file
def read1():
    '''
    读取并保存第一次
    '''
    file1=r'../data/atec_nlp_sim_train.csv'
    file2=r'../data/atec_nlp_sim_train_add.csv'
    t1=pd.read_csv(file1,index_col=None,header=None,encoding='utf-8',sep='\t')
    t2=pd.read_csv(file2,index_col=None,header=None,encoding='utf-8',sep='\t')
    train=pd.concat([t1,t2],axis=0)
    return train
#read1().to_csv('../data/nlp_train.csv',sep='\t',encoding='utf-8',header=None,index=None)

def read2():
    '''
    第一次读后的结果保存之后在读取
    '''
    file=r'../data/nlp_train.csv'
    train=pd.read_csv(file,sep='\t',encoding='utf-8',header=None,index_col=None)
    return train
train=read2()

* 1.1 字符串

In [3]:
def sent_append(df):
    '''
    把训练集的字符串结合起来。
    **莫名其妙出来一个\ufeff
    '''
    str_append=df.loc[:,1].append(df.loc[:,2]).reset_index(drop=True)
    return str_append.values
sentence=sent_append(train)

* 1.2分词

* 包含词性的分词

In [4]:
from jieba import posseg 
def cut_pseg(sentences):
    '''包含词性的分词
    '''
    jieba.add_word('花呗')
    jieba.add_word('借呗')
    sent_cut=[list(posseg.cut(i)) for i in sentences]
    word=[[j.word for j in i] for i in sent_cut]
    flag=[[j.flag for j in i] for i in sent_cut]
    return word,flag


* 普通的分词

In [5]:
def cut_sent(sentences):
    '''分词，用空格隔开
    '''
    jieba.add_word('花呗')
    jieba.add_word('借呗')
    sent_cut=[' '.join(jieba.cut(i,HMM=True)) for i in sentences]
    return sent_cut
sent_cut=cut_sent(sentence)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\customer\AppData\Local\Temp\jieba.cache
Loading model cost 1.604 seconds.
Prefix dict has been built succesfully.


* 把分词后的数据写入到文档中

In [6]:
sent_cut[0]

'\ufeff 怎么 更改 花呗 手机号码'

In [7]:
def write_sentences(sent):
    '''
    把数据写入文档中,需要先分词
    '''    
    with open('../data/sentences.txt','w',encoding='utf-8') as f:        
        for i in sent:
            f.write(i+'\n')
#write_sentences(sent_cut)

* 1.3 词向量

In [ ]:
import gensim
from gensim.models import word2vec

In [ ]:
model=word2vec.Word2Vec(sent_cut,min_count=1,iter=100)

* 保存

In [ ]:
model.save('../model/model')
model=word2vec.Word2Vec.load('../model/model')

* 利用以上的信息训练网络

####  一般特征
句子长度，
* 多个版本的数据

#### 5. 文档和文中对应的位置

In [ ]:
def get_vector()

## 3. 句向量
* doc2vec [参考](https://radimrehurek.com/gensim/models/doc2vec.html)

In [13]:
help(gensim.models.Doc2Vec)

Help on class Doc2Vec in module gensim.models.doc2vec:

class Doc2Vec(gensim.models.base_any2vec.BaseWordEmbeddingsModel)
 |  Class for training, using and evaluating neural networks described in http://arxiv.org/pdf/1405.4053v2.pdf
 |  
 |  Method resolution order:
 |      Doc2Vec
 |      gensim.models.base_any2vec.BaseWordEmbeddingsModel
 |      gensim.models.base_any2vec.BaseAny2VecModel
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, tag)
 |  
 |  __init__(self, documents=None, dm_mean=None, dm=1, dbow_words=0, dm_concat=0, dm_tag_count=1, docvecs=None, docvecs_mapfile=None, comment=None, trim_rule=None, callbacks=(), **kwargs)
 |      Initialize the model from an iterable of `documents`. Each document is a
 |      TaggedDocument object that will be used for training.
 |      
 |      Parameters
 |      ----------
 |      documents : iterable of iterables
 |          The `documents` iterable can be simply a list of Tag

### 3.1 直接读取文件

In [28]:
documents = gensim.models.doc2vec.TaggedLineDocument('../data/sentences.txt')

In [33]:
doc2vec_doc=gensim.models.Doc2Vec(documents,epochs=10000,window=4,sample=1e-5,workers=20,min_count=1)

* 保存模型

In [42]:
doc_name='../model/model_doc2vec'
doc2vec_doc.save(doc_name)
#doc2vec_doc = gensim.models.Doc2Vec.load(doc_name) 

* 如果不想训练，可以删掉训练文件

In [59]:
#model.delete_temporary_training_data(keep_doctags_vectors=True, \
#                                     keep_inference=True)

In [47]:
#计算句子的向量
#test_vec=doc2vec_doc.infer_vector(['借呗', '换网', '商贷'])


In [51]:
test_vec=doc2vec_doc.infer_vector([ '花呗', '账号','忘记'])
#计算与内容类似的文字
for i,j in doc2vec_doc.docvecs.most_similar([test_vec],topn=5):
    print(i,j,sent_cut[i])

121605 0.6982710361480713 花呗 忘记 密码
93559 0.6922953724861145 忘记 还 花呗
32808 0.6873430609703064 花呗 密码 忘记
143742 0.6861186027526855 忘记 花呗 密码
136644 0.6854676008224487 花呗 忘记 密码


In [13]:
def read_sentences():
    '''
    读取分割好的句子
    '''
    file='../data/sentences.txt'
    sentences=pd.read_table(file,encoding='utf-8',header=None)
    return sentences
sentences=read_sentences()

In [24]:
ranks=[]
second=[]
for tags,words in enumerate(sentences.values):
    vec=doc2vec_doc.infer_vector(words)
    similar=doc2vec_doc.docvecs.most_similar([vec],topn=len(sentences))
    rank=[doc for doc, sim in similar].index(tags)
    ranks.append(rank)
    second.append(similar[1])
    if tags%10000==0:
        print(tags)

0
10000
20000
30000
40000


KeyboardInterrupt: 

* doc2vec准确率判断
* 验证句子训练的效果。看看与自己最相似的是不是自己
* 看一下计算出来自己排在哪一个名词，如果是0自己与自己相似

### 3.2 自己构造tagged

In [ ]:
#构造自己想要的
train_corpus=[]
for tags,words in enumerate(sent_cut):
    train_corpus.append(gensim.models.doc2vec.TaggedDocument(words,[tags]))

In [ ]:
docvec=gensim.models.Doc2Vec(train_corpus,epochs=100)

## 4 关键字提取

In [ ]:
hb='为何我无法申请开通花呗信用卡收款'
def extra_tags_by_text(text,*arg):
    '''
    提取关键字并按照文章顺序返回。
    不能使用带权重和标签的返回值。 
    '''
    extra=jieba.analyse.extract_tags(text,*arg)
    index=[text.find(i) for i in extra ]
    index=np.argsort(index)
    extra=[extra[i] for i in index]
    return extra
print(extra_tags_by_text(hb))
hb2=extra_tags_by_text(hb)